# import def func

In [1]:
# import logging
import sys
# stdout=sys.stdout
sys.path.append('../../')
import os
import random
import time

# import matplotlib
# import matplotlib.pyplot as plt
# import mplfinance as mpf
# sys.stdout=stdout
import numpy as np
import pandas as pd
from JohnsonUtil import commonTips as cct
from JohnsonUtil import johnson_cons as ct
from JSONData import sina_data
from JSONData import tdx_data_Day as tdd
from JSONData import tdx_hdf5_api as h5a

# from mplfinance.original_flavor import candlestick_ohlc
# from JSONData import tdx_hdf5_api as h5a

# %matplotlib widget
# plt.rc('font', family='SimHei', size=13)
# matplotlib.get_backend()
# jupyter nbextension enable --py widgetsnbextension

os.environ['NUMEXPR_MAX_THREADS'] = '12'
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"
import nest_asyncio
nest_asyncio.apply()

In [139]:
def show_tdx_data(df, codelist=None, market_value='3', col=None,limit=20,orderby='percent'):
    col_src = [
        "name",'percent', "couts", "date", "perc", "ra_diff","xratio", "idx", "hat", "close",
        "volsum5d", "hmax", "lmin", "perc1d", "ma20", "ma250", "nclose",
        "llow", 'vol1d', 'vol2d', 'vol3d', 'vol4d', 'vol5d', "perc%sd" %
        (market_value),
        "perc5d"]

    if isinstance(df, pd.Series):
        col_name = [co for co in col_src if co in df.index]
        # col_name.insert(1,'date')
    else:
        col_name = [co for co in col_src if co in df.columns]

    # print(col_name)
    # if isinstance(df, pd.DataFrame) and 'name' in df.columns:
    #     columns_ = col_name
    # else:
    #     if isinstance(df, pd.Series) and 'name' in df.index:
    #         columns_ = col_name.insert(1, "date")
    #     else:
    #         columns_ = col_name.remove("name")
    if orderby in df.columns:
        df=df.sort_values('percent', ascending=False)
    if isinstance(col,list):
        col_name.extend(col)
    elif col is not None:
        col_name.insert(4, col)

    if codelist is None:
        if isinstance(df, pd.Series):
            print("Count:1")
            temp = df.loc[col_name][:limit]
        else:
            print("Count:%s %s" % (df.shape))
            temp = df.loc[:, col_name][:limit]
    else:
        temp = df.loc[[x for x in codelist if x in df.index], col_name][:limit]
    return temp

def clean_sina_dm(dd):
    if (cct.get_now_time_int() > 915 and cct.get_now_time_int() < 926):
        dd = dd[dd.close > 0]
    else:
        dd = dd[dd.open > 0]
    return dd

global sina_dm
sina_dm = None


def get_sina_data_cname(df, dm=None, index=False, realtime=False):
    # index_status=False
    global sina_dm
    if dm is None:
        if sina_dm is None:
            dm = sina_data.Sina().market('all')
        else:
            dm = sina_dm.copy()

    if 'ticktime' in df.columns:
        time_l = time.mktime(dm.loc[:, 'ticktime'][-1].timetuple())
        dur = round((time.time() - time_l) / 60, 2)
        if dur > 5:
            print("timel > 5,renew dm")
            dm = sina_data.Sina().market('all')
            sina_dm = dm.copy()

    if realtime:
        print("compute percent")
        df.rename(columns={'low': 'llow',
                  'high': 'lhigh', 'open': 'lopen'}, inplace=True)
        if 'nclose' in dm.columns:
            dd = cct.combine_dataFrame(
                df, dm.loc
                [:,
                 ['nclose', 'close', 'name', 'open', 'high', 'low', 'llastp',
                  'ticktime']],
                clean=True)
        else:
            dd = cct.combine_dataFrame(
                df, dm.loc
                [:, ['close', 'name', 'open', 'high', 'low', 'llastp', 'ticktime']],
                clean=True)
        dd['perc'] = ((dd['close'] - dd['llastp']) / dd['llastp'] *
                      100).map(lambda x: round(x, 2))
        # dd = cct.combine_dataFrame(df, dm['name'])

    else:
        dd = cct.combine_dataFrame(df, dm['name'])
    # cname = sina_data.Sina().get_code_cname(code)
    return dd

# from JSONData import tdx_hdf5_api as h5a
names = locals()
def get_tdx_all_now(resample='d',market_blk='all',showtable=True):
    top_now = tdd.getSinaAlldf(market=market_blk, vol=ct.json_countVol, vtype=ct.json_countType)
    if resample=='d' or resample=='3d':
        dl='60'
    elif  resample=='w':
        dl='200'
    else:
        dl='300'
    filter='y'
#     tdx_df = f'lastpTDX_DF_{resample}'
    if not f'lastpTDX_DF_{resample}' in names:
        print(f'lastpTDX_DF_{resample} not in locals')
        top_all, lastpTDX_DF = tdd.get_append_lastp_to_df(top_now, dl=dl,resample=resample,showtable=showtable)
        names[f'lastpTDX_DF_{resample}'] = lastpTDX_DF
    else:
        print(f'lastpTDX_DF_{resample} in locals')
        top_all = tdd.get_append_lastp_to_df(top_now, eval(f'lastpTDX_DF_{resample}'))
    return top_all

def get_tdx_all_from_h5_old(resample='d',showtable=True):
    #sina_monitor
    h5_fname = 'tdx_last_df'
    resample=resample
    if resample=='d' or resample=='3d':
        dl='60'
    elif  resample=='w':
        dl='200'
    else:
        dl='300'
    filter='y'
    h5_table = 'low' + '_' + resample + '_' + str(dl) + '_' + filter + '_' + 'all'
    h5 = h5a.load_hdf_db(h5_fname, table=h5_table,code_l=None, timelimit=False,showtable=showtable)
    if h5 is None:
        h5,tdx = tdd.get_append_lastp_to_df(sina_data.Sina().market('all'),dl=dl,resample=resample)
    return h5


# def get_tdx_all_from_h5_old(resample='d',dl=ct.duration_date_day, end=None, ptype='low', filter='y',showtable=True):
#     if end is not None:
#         h5_table = ptype + '_' + resample + '_' + str(dl) + '_' + filter + \
#             '_' + end.replace('-', '') + '_' + 'all'
#     else:
#         h5_table = ptype + '_' + resample + '_' + str(dl) + '_' + filter + '_' + 'all'
#     #sina_monitor
#     h5_fname = 'tdx_last_df'
#     resample=resample
#     if resample=='d' or resample=='3d':
#         dl='60'
#     elif  resample=='w':
#         dl='200'
#     else:
#         dl='300'
#     filter='y'
#     h5_table = 'low' + '_' + resample + '_' + str(dl) + '_' + filter + '_' + 'all'
#     h5 = h5a.load_hdf_db(h5_fname, table=h5_table,code_l=None, timelimit=False,showtable=showtable)
#     return h5

def write_code_to_blk(codew, blk='077',rewrite=False):
    block_path = tdd.get_tdx_dir_blocknew() + '%s.blk' % (blk)
    write_blk = 'n'
    if not rewrite:
        write_blk = cct.cct_raw_input("write blk [Y] or [N]:")
        if write_blk == 'y' or write_blk == 'Y':
            hdf5_wri = cct.cct_raw_input(
                "Rewrite code [Y] or append [N](defalut:N):")
            if hdf5_wri == 'y' or hdf5_wri == 'Y':
                append_status = False
            else:
                append_status = True
            if len(codew) > 3:
                cct.write_to_blocknew(block_path, codew, append=append_status,
                                      doubleFile=False, keep_last=0)
                print("write:%s block_path:%s" % (len(codew), block_path))
            else:
                print("write error:%s block_path:%s" % (len(codew), block_path))
    else:
        if len(codew) > 3:
                cct.write_to_blocknew(block_path, codew, append=False,
                                      doubleFile=False, keep_last=0)
                print("write:%s block_path:%s" % (len(codew), block_path))
        else:
            print("write error:%s block_path:%s" % (len(codew), block_path))

def apply_col_toint(df, col=None):
    if col is None:
        co2int = ['boll', 'op', 'ratio', 'fib', 'fibl', 'df2']
    # co2int.extend([co for co in df.columns.tolist()
    #                if co.startswith('perc') and co.endswith('d')])
        co2int.extend(['top10', 'topR'])
    else:
        co2int = col
    co2int = [inx for inx in co2int if inx in df.columns]

    for co in co2int:
        df[co] = df[co].astype(int)

    return df

def readHdf5(fpath, root=None):
    if not os.path.exists(fpath):
        print(f'fpath is not exists')
        return None
    store = pd.HDFStore(fpath, "r")
    print(store.keys())
    if len(store.keys()) > 0 and root is None and store is not None:
        root = store.keys()[0].replace("/", "")
        df = store[root]
        df = apply_col_toint(df)
    else:
        df = None
    store.close()
    return df

In [3]:
f'lastpTDX_DF_d' in names
f'lastpTDX_DF_d' in locals()
# eval(f'lastpTDX_DF_d')

False

False

## init data

### get_tdx_all

In [4]:
# h5_day=get_tdx_all_now(resample='d')

### sina

### sina_all

In [5]:
sina_dm = sina_data.Sina().market('all')
lastday = cct.last_tddate()
lastday,sina_dm.name[:1]
sina_dm = clean_sina_dm(sina_dm)

('2025-06-20',
 code
 603031    安孚科技
 Name: name, dtype: object)

In [6]:
cct.get_time_to_date(time.time(), format='%Y-%D %H:%M:%S')

# abc=dm.loc[:,'ticktime'][-1]
sina_dm.ticktime[-2:]
# dm.ticktime[:1]
# sina_dm.loc[:, 'ticktime'][-1]
# time.strptime(abc,format='%Y-%m-%d %H:%M:%S')
# (time.mktime(abc.timetuple()))
(time.time() - time.mktime(sina_dm.loc[:, 'ticktime'][-1].timetuple())) / 60

'2025-06/22/25 08:30:44'

code
002408   2025-06-20 15:00:00
002449   2025-06-20 15:00:00
Name: ticktime, dtype: datetime64[ns]

2490.7472658991815

#### 实时成交

In [7]:
# tdx_path = "G:\\tdx_last_df.h5"
# from JohnsonUtil import commonTips as cct
sina_MultiD_path = f"{cct.get_ramdisk_dir()}\\sina_MultiIndex_data.h5".replace('\\',os.sep)
print(f'sina_MultiD_path:{sina_MultiD_path}')
h5 = readHdf5(sina_MultiD_path)
print(sina_MultiD_path)
if h5 is not None:
    h5.shape
    h5code='603019'
    h5.loc[h5code][-3:]
    code_muti = '603019'
    # h5.loc[code_muti][:2]

    freq = 'D'
    # freq = '5M'
    startime = '09:25:00'
    endtime = '15:01:00'

    time_ratio = cct.get_work_time_ratio()
    time_ratio
    run_col = ['close', 'volume']
    # run_col = ['close']
    mdf = cct.get_limit_multiIndex_freq(
        h5, freq=freq.upper(),
        col=run_col, start=startime, end=endtime, code=None)
    allvol = mdf.volume.sum()
    allvol = round(allvol / 10 / 1000 / 1000,2)
    print("总量:%s 亿"%(allvol))
    mdf['vol']=mdf['volume'].apply(lambda x: round(x /1000/1000,2)) 
    mdf['amount']=(mdf.volume * mdf.close).map(lambda x: round(x/100/1000/1000, 2))
    #全局占比
    mdf['volume'] = mdf.amount.apply(
        lambda x: round(x / allvol * 100 / time_ratio, 2))
    mdf=mdf.reset_index().set_index('code')
    mdf.loc[h5code]
    str(h5.index.get_level_values(1)[-1])[:10]
else:
    mdf = None

sina_MultiD_path:/Volumes/RamDisk/sina_MultiIndex_data.h5
['/all_30']
/Volumes/RamDisk/sina_MultiIndex_data.h5


(2767032, 6)

,close,high,low,llastp,volume,lastbuy
ticktime,,,,,,
2025-06-20 14:58:58,66.9,67.88,66.75,67.55,16223684,67.11
2025-06-20 14:59:40,66.9,67.88,66.75,67.55,16223684,67.18
2025-06-20 15:00:01,66.9,67.88,66.75,67.55,16398684,67.07


1.0

总量:8715.97 亿


ticktime    2025-06-21 00:00:00
close                  67.21773
volume                     0.13
vol                        16.4
amount                    11.02
Name: 603019, dtype: object

'2025-06-20'

#### 读取实时成交个股

In [8]:
if mdf is not None:
    nowdf = mdf.sort_values(by=['volume'], ascending=[0])
    # nowdf.shape, df.shape, code_idx.shape
    # top20_nd = nowdf.reset_index().loc[:, ['code', 'volume']][:20]
    top20_nd = nowdf.reset_index().loc[:, ['code', 'volume']][:20]
    top20_nd = get_sina_data_cname(top20_nd)
    nowdf.shape, top20_nd

((5420, 5),
         volume   name
 code                 
 300308    0.73   中际旭创
 300468    0.71   四方精创
 600519    0.57   贵州茅台
 300476    0.55   胜宏科技
 300502    0.54    新易盛
 601318    0.53   中国平安
 300773    0.49    拉卡拉
 002475    0.45   立讯精密
 002463    0.45   沪电股份
 002104    0.43   恒宝股份
 000858    0.41  五 粮 液
 300059    0.39   东方财富
 300750    0.38   宁德时代
 003040    0.36    楚天龙
 002384    0.35   东山精密
 002017    0.33   东信和平
 603300    0.32   海南华铁
 300255    0.31   常山药业
 002050    0.31   三花智控
 002594    0.29    比亚迪)

##### 合并实时成交量及SinaDM

In [9]:
# mdf[:1]
if mdf is not None:
    sinadm = cct.combine_dataFrame(sina_dm, mdf[['vol','amount']])
else:
    sinadm = sina_dm
sinadm['percent']=((sinadm['close']-sinadm['llastp'])/(sinadm['llastp'])*100).round(1)


## resample 多周期

### filterKey

In [114]:
filter_query={'2周期上冲上轨And涨幅新高': 'lasth1d > upper1 and lasth2d >upper2 and high >boll and percent > maxp',
             '3周期新高上轨':'lastp1d > lastp2d > lastp3d and lasth1d > upper1 and lasth2d > upper2 and lasth3d > upper3',
             '3周期新高大于Ma5': 'lastp1d > lastp2d and lasth2d > lasth3d and lastl2d > lastl3d and lastp1d > ma51d',
             '新高上upper':'lasth1d  >lasth2d  and lastp1d > ma51d and lastp1d > lastp2d*0.99 and lasth2d > lasth3d  and lasth1d >upper',
            }   
filter_keys=list(filter_query.keys())

In [115]:
def get_filter(idx=0):
    filter_key=filter_query[filter_keys[idx]]
    print(f'{filter_keys[idx]}: {filter_key}')
    return filter_key

In [116]:
filterIDX=3
# h5_3d=None,h5_d=None,h5_w=None,h5_m=None

### resample 3d

In [117]:
# h5_3d = get_tdx_all_from_h5(resample='3d') if h5_3d is None else h5_3d
h5_3d = get_tdx_all_now(resample='3d')
# tdx_3d = cct.combine_dataFrame(h5_3d,sinadm)

initdx :0 b1>:5180 it:0.5 lastpTDX_DF_3d in locals


In [118]:
# tdx_now.query('close >  lastp1d  and lastl1d < ma51d and close > ma51d and lastp1d < upper and lastp1d > lastp2d and lasth1d >high4 and boll > 0 and volume < 1')
# tdx_select = tdx_3d.query('close > lastp1d and percent < 3')
# tdx_select = tdx_3d.query('lastp1d > lastp2d > lastp3d and lasth1d > upper1 and lasth2d > upper2 and lasth3d > upper3')
# tdx_select = tdx_3d.query('lastp1d > lastp2d  and lasth2d > lasth3d and (lasth2d > upper2 or lasth3d > upper3) and lasth2d < hmax')
# tdx_select = tdx_3d.query('lastp1d > lastp2d  and lasth2d > lasth3d and lasth3d > lasth4d and lastl1d > lastl2d and lastl2d > lastl3d')
# t3d_code = h5_3d.query(f'{get_filter(filterIDX)}')
t3d_code = h5_3d.query('lasth1d > ma51d and lasth2d > ma52d and lasth3d > ma53d and ma51d > ma201d and ma52d >ma202d and ma53d > ma203d and macd > 0 and macddif > macddea')
show_tdx_data(t3d_code,limit=10)

Count:2137 221


,name,couts,date,close,hmax,lmin,perc1d,nclose,llow,perc3d,perc5d
code,,,,,,,,,,,
603031,安孚科技,0,2025-04-08,27.80,29.31,23.55,0.0,27.83,24.12,-0.10,0.1
301093,华兰股份,0,2025-04-08,31.11,31.73,21.52,1.2,31.09,22.20,-0.10,1.2
301331,恩威医药,0,2025-04-08,29.50,31.42,21.45,-1.1,28.81,21.92,25.91,1.3
688578,艾力斯,0,2025-04-08,92.50,97.62,75.19,-1.1,92.62,78.62,0.00,-1.1
600661,昂立教育,0,2025-04-08,9.98,10.56,8.43,-1.1,10.02,8.92,-1.10,0.1
839790,联迪信息,0,2025-04-08,38.53,43.69,28.58,-1.1,39.38,28.58,0.00,1.1
301170,锡南科技,0,2025-04-08,24.62,26.00,18.16,-1.1,24.78,19.62,1.20,0.1
300400,劲拓股份,0,2025-04-08,16.19,16.27,11.91,0.2,15.89,12.66,-0.10,1.2
301588,美新科技,0,2025-04-08,18.56,18.95,13.66,1.3,18.48,14.80,0.10,1.3


In [155]:
if '301038' in t3d_code.index:
    h5_3d.loc['301038',['lastdu4','boll','ral','fib','percmax','macd','macddif','macddea','macdlast2','macdlast3','lasth1d','lastl1d','lastl2d','lastl3d','lastp1d','ma51d','ma52d','ma53d','ma54d']]

lastdu4       1.05
boll             0
ral             10
fib              0
percmax        5.0
macd          0.34
macddif      -0.06
macddea      -0.23
macdlast2     0.43
macdlast3     0.47
lasth1d      14.82
lastl1d      13.92
lastl2d      14.67
lastl3d      14.39
lastp1d      14.38
ma51d        14.59
ma52d        14.61
ma53d         14.6
ma54d         14.6
Name: 301038, dtype: object

In [120]:
# t3d_code = tdx_3d.query('lasth1d > lasth2d > lasth3d and close >= high4')
# t3d_code = h5_3d.query(f'{get_filter(1)}')
# show_tdx_data(t3d_code)

### resample W

In [121]:
# h5_w = get_tdx_all_from_h5(resample='w') if h5_w is None else h5_w

h5_w = get_tdx_all_now(resample='w') 
# tdx_w = cct.combine_dataFrame(h5_w,sinadm)

initdx :0 b1>:5180 it:0.4 lastpTDX_DF_w in locals


In [156]:
# tw_code = tdx_w.query('lasth1d > lasth2d > lasth3d and close >= high4')
# tw_code = tdx_w.query('lasth1d > lasth2d and lasth2d >lasth3d and lasth2d > upper1')
# tw_code = tdx_w.query ('lastp1d > lastp2d and lasth2d > lasth3d and lastl2d > lastl3d and lastp1d > ma51d')
# tw_code = tdx_w.query(f'{get_filter(0)}')
# tw_code = tdx_w.query(f'{get_filter(filterIDX)}')
# tw_code = h5_w.query(f'{get_filter(filterIDX)}')
tw_code = h5_w.query('lastp2d > ma51d and lastp3d > ma51d and macd > 0 and macddif > macddea')
show_tdx_data(tw_code,limit=10)

Count:1244 223


,name,percent,couts,date,close,hmax,lmin,perc1d,nclose,llow,perc3d,perc5d
code,,,,,,,,,,,,
301038,深水规院,20.09,15,2025-01-03,17.27,17.41,11.53,-1.0,16.09,12.98,0.10,1.20
300410,正业科技,19.97,71,2025-04-18,7.45,7.09,4.00,0.0,7.43,4.67,27.30,0.00
688117,圣诺生物,15.04,7,2025-03-07,35.72,26.27,15.78,-0.9,34.84,15.93,24.20,1.20
603616,韩建河山,10.12,24,2025-04-11,4.46,4.54,2.76,-0.1,4.26,2.76,29.70,1.10
603209,兴通股份,10.04,8,2025-04-18,16.44,16.53,12.93,0.0,16.27,13.75,26.09,0.10
603022,新通联,10.02,0,2025-01-03,9.33,8.76,6.46,-1.1,9.20,6.80,23.70,1.20
603948,建业股份,9.98,0,2025-04-18,19.61,18.59,14.20,-1.1,19.12,15.90,0.00,0.10
002846,英联股份,9.98,24,2025-01-27,13.55,11.61,7.68,0.0,13.54,7.97,0.00,18.91
688093,世华科技,7.95,1,2025-01-10,28.91,26.56,16.84,1.2,27.89,17.25,27.30,-0.10


In [123]:
if '301038' in tw_code.index:
    h5_w.loc['301038',['macd','macddif','macddea','macdlast2','macdlast3','lasth1d','lastl1d','lastl2d','lastl3d','lastp1d','ma51d','ma52d','ma53d','ma54d']]

macd          0.21
macddif      -0.11
macddea      -0.22
macdlast2     0.24
macdlast3     0.16
lasth1d       15.0
lastl1d      13.92
lastl2d      14.17
lastl3d      14.23
lastp1d      14.38
ma51d        14.52
ma52d        14.39
ma53d        14.18
ma54d        13.91
Name: 301038, dtype: object

### resample M

In [124]:
# h5_m = get_tdx_all_from_h5(resample='m') if h5_m is None else h5_m
h5_m = get_tdx_all_now(resample='m')
# tdx_m = cct.combine_dataFrame(h5_m,sinadm)

initdx :0 b1>:5180 it:0.4 lastpTDX_DF_m in locals


In [125]:
# tm_code = tdx_m.query('lastp1d > lastp2d and lasth2d > lasth3d and lastp1d > ma51d')
# tm_code = tdx_m.query('lastp1d > lastp2d and lasth2d > lasth3d and lastl2d > lastl3d and lastp1d > ma51d')
#月线站上,3月反弹,2月新高
tm_code = h5_m.query('lastl1d > lastl2d > lastl3d and lasth1d > lastp2d > lastp3d and  ma51d > ma52d and macd > 0 and macddif > macddea')   
# tm_code = h5_m.query(f'{get_filter(filterIDX)}')
show_tdx_data(tm_code,limit=10)

Count:1975 221


,name,couts,date,close,hmax,lmin,perc1d,nclose,llow,perc3d,perc5d
code,,,,,,,,,,,
301093,华兰股份,0,2024-10-31,31.11,28.08,17.12,1.10,31.09,21.78,0.0,1.20
301331,恩威医药,0,2024-12-31,29.50,27.16,20.06,30.59,28.81,22.83,0.0,1.20
837023,芭薇股份,0,2024-09-30,18.62,14.39,6.22,-0.90,18.78,6.22,0.0,44.50
688578,艾力斯,0,2024-10-31,92.50,84.88,39.41,1.20,92.62,49.59,1.3,27.09
839790,联迪信息,0,2024-09-30,38.53,42.47,6.12,1.20,39.38,6.55,-1.0,1.20
837006,晟楠科技,0,2024-09-30,34.09,29.33,8.68,1.20,34.66,8.68,1.2,0.00
688679,通源环境,0,2024-09-30,13.66,10.52,5.86,-0.90,13.54,5.86,0.0,1.20
003016,欣贺股份,0,2025-04-30,7.14,9.29,5.78,-0.80,7.17,5.72,-0.1,1.10
301323,新莱福,0,2024-09-30,44.03,42.78,25.20,35.19,44.84,26.56,-1.1,1.20


In [126]:
if '301038' in tm_code.index:
    h5_m.loc['301038',['lasth1d','lastl1d','lastl2d','lastl3d','lastp1d','ma51d','ma52d','ma53d','ma54d']]
    
# h5_m.loc['301038',['lasth1d','lastp2d','lastp3d','lastp1d','ma51d','ma52d','ma53d','ma54d']]    

lasth1d    15.53
lastl1d    13.92
lastl2d    13.39
lastl3d    11.53
lastp1d    14.38
ma51d      14.13
ma52d      13.97
ma53d      13.92
ma54d      14.48
Name: 301038, dtype: object

### resample D

In [127]:
# h5_d = get_tdx_all_from_h5(resample='d') if h5_d is None else h5_d
h5_d = get_tdx_all_now(resample='d')
# tdx_d = cct.combine_dataFrame(h5_d,sinadm)

initdx :0 b1>:5180 it:0.4 lastpTDX_DF_d in locals


In [170]:
# td_code = tdx_d.query('lasth1d > lasth2d > lasth3d and close >= high4')
# td_code = tdx_d.query('ma51d > ma10d and ral > 3 and lastp1d >ma20d and low > lastl1d and vol > 2' )
# td_code = tdx_d.query('lastp1d > lastp2d  and lasth1d > lasth2d and lasth2d > lasth3d and lastl1d > lastl2d and lastl2d > lastl3d and close > open')

# td_code = h5_d.query(f'{get_filter(filterIDX)}')
# td_code = h5_d.query(f'lasth1d > lasth2d and close > lastp1d and ral > 10 and fib > 0 and boll > 0')
td_code = h5_d.query(f'lastv1d > lastv2d and close > lastp1d and ral > 10 and fib > 0 and boll > 0 and close > lastp2d and percent > per2d and percent > lastdu4 \
and close >high4 and close > max5 and (low > open*0.99 or (per1d > 4 and low >= lastp1d))')

show_tdx_data(td_code,limit=20)
if len(td_code) < 30:
    write_code_to_blk(td_code.sort_values('percent', ascending=False).index,'065',rewrite=True)

Count:24 224


,name,percent,couts,date,close,hmax,lmin,perc1d,nclose,llow,perc3d,perc5d
code,,,,,,,,,,,,
301038,深水规院,20.09,15,2025-04-07,17.27,15.36,13.51,0.20,16.09,11.53,-0.10,-0.90
301292,海科新源,20.06,13,2025-04-07,23.52,15.62,14.50,48.09,23.52,11.46,28.09,-0.10
300485,赛升药业,19.97,70,2025-04-07,13.94,13.89,7.99,-1.10,13.83,7.03,-1.10,33.19
300410,正业科技,19.97,71,2025-04-07,7.45,6.21,5.51,1.20,7.43,4.12,1.20,-1.10
301150,中一科技,18.56,15,2025-04-07,21.91,17.03,15.16,37.31,20.45,12.35,1.30,-1.10
839719,宁新新材,15.72,0,2025-04-07,23.70,17.45,11.23,1.20,22.64,8.90,0.00,1.20
603209,兴通股份,10.04,8,2025-04-08,16.44,15.24,14.13,-0.10,16.27,13.46,0.10,0.20
301217,铜冠铜箔,6.43,21,2025-04-07,12.08,11.16,10.22,1.30,12.04,8.49,0.30,-0.10
600288,大恒科技,5.34,28,2025-04-08,9.67,9.34,8.77,-0.10,9.54,7.54,0.10,-1.10


all write to /Users/Johnson/Documents/Johnson/WinTools/new_tdx2/T0002/blocknew/065.blk:24
all write to /Users/Johnson/Documents/Johnson/WinTools/new_tdx/T0002/blocknew/065.blk:24
write:24 block_path:/Users/Johnson/Documents/Johnson/WinTools/new_tdx2/T0002/blocknew/065.blk


In [150]:
if '301038' in td_code.index:
    h5_d.loc['301038',['lastdu4','close','boll','ral','fib','percmax','macd','macddif','macddea','macdlast1','macdlast2','macdlast3','lasth1d','lastl1d','lastl2d','lastl3d','lastp1d','ma51d','ma52d','ma53d','ma54d']]

lastdu4       1.07
close        17.27
boll             1
ral             16
fib              1
percmax        5.0
macd         -0.11
macddif       0.12
macddea       0.18
macdlast1    -0.11
macdlast2    -0.08
macdlast3    -0.02
lasth1d      14.82
lastl1d      14.22
lastl2d      13.92
lastl3d      14.28
lastp1d      14.38
ma51d        14.57
ma52d        14.76
ma53d        14.82
ma54d        14.79
Name: 301038, dtype: object

#### filter test

In [148]:
# h5_t_code=h5_w.query('lasth1d > upper1 and lasth1d >lasth2d   and high >boll and percent > maxp')
# h5_t_code=h5_d.query('lasth1d  >lasth2d  and lastp1d > ma51d and lastp1d > lastp2d*0.99 and lasth2d > lasth3d and lastp2d < upper2 and lasth1d >upper')
code_3f=  list(set(tm_code.index) & set(tw_code.index) & set(t3d_code.index))

show_tdx_data(t3d_code.loc[code_3f],limit=5)

Count:527 221


,name,percent,couts,date,close,hmax,lmin,perc1d,nclose,llow,perc3d,perc5d
code,,,,,,,,,,,,
301038,深水规院,20.09,15,2025-04-08,17.27,15.24,11.53,-1.1,16.09,11.53,30.3,0.0
300410,正业科技,19.97,71,2025-04-08,7.45,6.12,4.00,1.3,7.43,4.12,-1.1,1.3
688117,圣诺生物,15.04,7,2025-03-28,35.72,33.94,17.28,-1.1,34.84,17.28,-0.9,0.1
603616,韩建河山,10.12,24,2025-04-08,4.46,4.24,2.76,-1.0,4.26,2.94,-1.1,1.3
002846,英联股份,9.98,24,2025-04-08,13.55,13.63,7.81,-1.1,13.54,7.89,-0.9,12.3


In [131]:
# td_code2 = tdx_d.query(f'{get_filter(2)}')
# td_code2 = h5_d.query('lastp1d > lastp2d  and lasth1d > lasth2d and lasth2d > lasth3d and lastl1d > lastl2d and lastl2d > lastl3d and close > open')
# show_tdx_data(td_code2,limit=10)


#### 多周期筛选

In [164]:
for co in ['tm_code','tw_code','t3d_code','td_code']:
    print(f'{co}:{len(eval(co))}')
code_f=  list(set(tm_code.index) & set(tw_code.index) & set(t3d_code.index)  & set(td_code.index))
# code_f=  list(set(tm_code.index) & set(tw_code.index) & set(td_code.index))
code_f =td_code.loc[code_f].sort_values('percent', ascending=False).index.tolist()
if len(code_f) > 0:
    show_tdx_data(td_code.loc[code_f],limit=10)
else:
    print('no code')

tm_code:1975
tw_code:1244
t3d_code:2137
td_code:43
Count:7 224


,name,percent,couts,date,close,hmax,lmin,perc1d,nclose,llow,perc3d,perc5d
code,,,,,,,,,,,,
301038,深水规院,20.09,15,2025-04-07,17.27,15.36,13.51,0.20,16.09,11.53,-0.1,-0.9
300410,正业科技,19.97,71,2025-04-07,7.45,6.21,5.51,1.20,7.43,4.12,1.2,-1.1
605389,长龄液压,3.89,0,2025-04-08,39.25,35.53,31.20,34.69,39.19,25.70,1.3,1.3
300816,艾可蓝,3.07,1,2025-04-07,35.62,34.56,29.45,-1.00,35.81,24.34,1.3,-0.9
601555,东吴证券,2.57,146,2025-04-07,8.77,8.76,7.57,0.00,8.75,6.77,1.2,0.1
601336,新华保险,2.37,4,2025-04-07,56.56,55.72,49.00,0.00,56.16,46.34,1.3,0.1
300739,明阳电路,2.29,11,2025-04-07,14.28,13.67,12.52,1.20,14.23,10.37,0.1,-1.1


#### check data

In [133]:
'300998' in code_f
code_check = [co for co in code_f if co in h5_3d.index]
len(code_check)

False

9

In [134]:
show_tdx_data(td_code.loc[code_f])

Count:9 224


,name,couts,date,close,hmax,lmin,perc1d,nclose,llow,perc3d,perc5d
code,,,,,,,,,,,
301038,深水规院,15,2025-04-07,17.27,15.36,13.51,0.20,16.09,11.53,-0.1,-0.9
300410,正业科技,71,2025-04-07,7.45,6.21,5.51,1.20,7.43,4.12,1.2,-1.1
688313,仕佳光子,45,2025-04-07,38.59,36.62,27.20,-0.80,38.25,13.68,0.0,1.2
605389,长龄液压,0,2025-04-08,39.25,35.53,31.20,34.69,39.19,25.70,1.3,1.3
688331,荣昌生物,0,2025-03-28,62.81,71.00,42.19,-1.10,61.88,38.97,-1.1,-0.9
300739,明阳电路,11,2025-04-07,14.28,13.67,12.52,1.20,14.23,10.37,0.1,-1.1
688498,源杰科技,0,2025-04-07,166.75,156.00,127.25,1.20,165.00,92.00,0.1,0.1
002104,恒宝股份,377,2025-03-31,14.70,13.37,7.05,1.20,14.62,6.42,22.3,0.0
688565,力源科技,3,2025-04-07,7.95,8.60,7.27,-1.10,7.99,5.67,0.0,-0.1


### Write toblk

In [135]:
# write_code_to_blk(code_f,'077',rewrite=False)